# Transcribe

## 1. Package 설치

---------------------------------------

In [ ]:
# !pip install -U pip
# !pip install -U boto3

In [ ]:
%load_ext autoreload
%autoreload 2

# External Dependencies:
import time
import boto3
import json
import pandas as pd
import util.xer_py3 as xer
import IPython.display as ipd

transcribe = boto3.client('transcribe')
s3 = boto3.resource('s3')

## 2. 환경 설정

---------------------------------------

In [ ]:
## 실험
job_name = "XXXXXXX"  ## Job 이름

bucket_name ='XXXXXXX'  ## CloudFormation의 ouput에서 나온 S3Bucket
prefix = 'XXXXXXX' ## S3 bucket 내 음성파일을 upload 한 폴더 명
media_filename = 'XXXXXX.wav'  ##  wav | flac | mp3 | mp4 

MediaFileUri = "s3://{}/{}/{}".format(bucket_name, prefix, media_filename)
OutputBucketName = bucket_name
MediaFormat = media_filename.split('.')[1]

%store bucket_name

In [ ]:
from botocore.exceptions import ClientError
s3_client = boto3.client('s3')

region_name=boto3.session.Session().region_name

try:
    s3_client = boto3.client('s3', region_name=region_name)
    location = {'LocationConstraint': region_name}
    s3_client.create_bucket(Bucket=bucket_name,
                            CreateBucketConfiguration=location)
except ClientError as e:
    print(e)

## 3. Transcribe 시작

---------------------------------------

 - **MediaFileUri** : S3 내 미디어 파일 위치
 - **MediaSampleRateHertz** : 입력 파일 RateHertz 설정, 미설정 시 자동 인식
 - **MediaFormat** : 'mp3'|'mp4'|'wav'|'flac'
 - **LanguageCode** : ko-KR 언어 설정
 - **OutputBucketName** : output 버킷 설정, 미설정 시 Service managed S3 bucket에 저장 90일 내 삭제 (job 이 삭제 되기 때문임)
 - **Settings** :
    > **VocabularyName** : 고객 단어 사전 (사전 등록 필요)   
    > **VocabularyFilterName** : 삭제가 필요한 단어 (사전 등록 필요)   
    > **VocabularyFilterMethod** : remove (삭제) | mask (```***``` 로 대체)   
    > **ShowAlternatives** : 대체 단어 추천 유무 (True|False)  
    > **MaxAlternatives** : 대체 단어 추천 수 (2~ 10까지 가능)  

    > 하나만 선택 가능
    > 1. 화자 식별
    >     > **ShowSpeakerLabels** : True | False   
    >     > **MaxSpeakerLabels** : 최대 식별 화자 수 ( 2~ 10까지 가능)
    > 2. 채널 식별
    >     > **ChannelIdentification** : True | False  

<img src="./images/transcribe_limit.png" width="600" height="400">


---------------------------------------


In [ ]:
try:
    transcribe.delete_transcription_job(TranscriptionJobName=job_name)
except:
    pass

In [ ]:
%%time
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': MediaFileUri},
#     MediaSampleRateHertz=44100,
    MediaFormat=MediaFormat,
    LanguageCode='ko-KR', ## en-US, ko-KR
    OutputBucketName=OutputBucketName,
    Settings={
        'VocabularyName': VocabularyName, 
#         'ShowSpeakerLabels': True,
#         'MaxSpeakerLabels': 2,
#         'ChannelIdentification': True,
#         'ShowAlternatives': True,
#         'MaxAlternatives': 3,
#         'VocabularyFilterName': 'test-remove-voca',
#         'VocabularyFilterMethod': 'mask'
    },
#     ContentRedaction={
#         'RedactionType': 'PII',
#         'RedactionOutput': 'redacted'|'redacted_and_unredacted'
#     }
)

while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

## 4. 결과 확인

---------------------------------------

In [ ]:
response = transcribe.get_transcription_job(
    TranscriptionJobName=job_name
)
print(response)

### Output json 파일 정보

In [ ]:
res = response['TranscriptionJob']['Transcript']['TranscriptFileUri']

In [ ]:
tmp=res.split('/')
bucket = tmp[3]
output_filename = tmp[4]
output_path = './output/' + output_filename
print("bucket : {}, output_filename : {}, output_path : {}".format(bucket, output_filename, output_path))

### From S3 파일 to SageMaker Notebook 복사

In [ ]:
s3.Object(bucket, output_filename).download_file(output_path)

In [ ]:
with open(output_path, 'rt', encoding='UTF8') as f:
    print(f)
    content = json.load(f)
print(content)

### STT 결과

In [ ]:
result = content['results']
res_item = list(result.keys())
transcript = result['transcripts'][0]['transcript']
print(transcript)

### 결과 상세 확인

In [ ]:
full_items = []
items =result['items']

for item in items:
    if 'start_time' in item:
        start_time = item['start_time']
        end_time = item['end_time']
        full_items.append([start_time, end_time, item['alternatives'][0]['content'], item['alternatives'][0]['confidence']])
    else:
        full_items.append([end_time,end_time,item['alternatives'][0]['content'],item['alternatives'][0]['confidence']])

df = pd.DataFrame(full_items, columns=['start_time', 'end_time', 'content', 'confidence']) 

## 동일 결과 삭제
df = df.drop_duplicates()
df = df.drop_duplicates(['start_time','content','confidence'], keep='first')

df['start_time'] = df['start_time'].astype('float')
df['end_time'] = df['end_time'].astype('float')

df=df.sort_values(by='start_time')
df

In [ ]:
print(transcript)
%store transcript

## 6. 결과 개선
-------------------------------

 - PCM 16비트로 녹음된 무손실 형식(예: FLAC 또는 WAV)을 사용합니다.
 - 저품질 오디오에는 8000Hz를 사용하고 고품질 오디오에는 16000Hz를 사용합니다.

2가지 방식으로 결과 개선이 가능합니다.
 - Custom vocabulary 생성  
 > - 도메인 특화 단어, 구, 단어 생성 가능  
 > - account 당 100 개 vocabulary 까지 가능  
 > - 사이즈는 최대 50KB 가능
 > - 단어 리스트로 추가하거나, 테이블 방식으로 추가 가능
 
 - Vocabulary filtering 생성  
 > - 마스크 또는 제거 방식 둘 중 선택 가능
 > - 사이즈는 최대 50KB 가능  

### Custom vocabulary 생성
--------------------------
각 단어를 한 줄씩 배치하거나 단어나 구절을 쉼표로 서로 분리하여 한 줄에 여러 단어를 배치할 수도 있습니다.
각 항목은 다음과 같이 구성되어야 합니다.
- 특정 단어 또는 구절을 대상으로 사용하기에 가장 적합
- 256자 미만(하이픈 포함)
- 허용되는 문자 집합의 문자만 가능 (한국어 가능)
- 계정당 최대 100개의 어휘 보유 가능
- 사용자 지정 어휘의 크기 한도는 50Kb
> Los-Angeles  
> F.B.I.  
> Etienne  
> Los-Angeles, F.B.I., Etienne



- 영어 : https://docs.aws.amazon.com/transcribe/latest/dg/charsets.html#char-english
- 한국어 : https://docs.aws.amazon.com/transcribe/latest/dg/charsets.html#char-korean
##### 한국어의 경우 링크 내 있는 IPA Character는 동작하지만, 인터넷의 IPA는 동작하지 않습니다.

In [ ]:
custom_file_path = './custom-vocabulary-example/custom-vocabulary-korean.txt'
custom_file_name = custom_file_path.split("/")[-1]
custom_voca_uri = f"s3://{bucket_name}/custom-voca/{custom_file_name}"

In [ ]:
!aws s3 cp ./custom-vocabulary-example/$custom_file_name $custom_voca_uri

In [ ]:
VocabularyName = 'custom-voca-ko'

try:
    transcribe.delete_vocabulary(VocabularyName=VocabularyName)
except:
    pass
finally: ## update_vocabulary() 도 추가로 있음
    add_voca = transcribe.create_vocabulary(
        VocabularyName=VocabularyName, ## Custom Vocabulary 이름 (case-sensitive)
        LanguageCode='ko-KR',  ## 'ko-KR' en-US
        VocabularyFileUri=custom_voca_uri
    )

while True:
    status = transcribe.get_vocabulary(VocabularyName=VocabularyName)
    if status['VocabularyState'] in ['READY', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

In [ ]:
add_voca = transcribe.create_vocabulary(
    VocabularyName='XXXXXXX', ## Custom Vocabulary 이름 (case-sensitive)
    LanguageCode='ko-KR',  ## 'ko-KR' en-US
    Phrases=[
        '엘레', ## 예시
        'XXX'
    ],
)

### Vocabulary filtering 생성

In [ ]:
remove_voca = transcribe.create_vocabulary_filter(
    VocabularyFilterName='XXXXXX', 
    LanguageCode='ko-KR',
    Words=[
        '스포츠',  ## 예시
    ],
#     VocabularyFilterFileUri='string'
)